In [0]:
%sql
SELECT * FROM pyspark_cata.source.products

id,name,price,category,updatedDate
1,iPhone,1000,electronics,2025-10-26T02:46:53.245Z
2,Macbook,2000,electronics,2025-10-26T02:46:53.245Z
3,T-Shirt,10,clothing,2025-10-26T02:46:53.245Z
4,Shirt,20,clothing,2025-10-26T02:46:53.245Z
5,Pants,30,clothing,2025-10-26T02:46:53.245Z


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

In [0]:
df = spark.sql("select * from pyspark_cata.source.products")

#Dedup
df = df.withColumn("dedup",row_number().over(Window.partitionBy("id").orderBy(desc("updatedDate"))))
df = df.filter(col("dedup") == 1).drop("dedup")
display(df)

id,name,price,category,updatedDate
1,iPhone,1000,electronics,2025-10-26T02:46:53.245Z
2,Macbook,2000,electronics,2025-10-26T02:46:53.245Z
3,T-Shirt,10,clothing,2025-10-26T02:46:53.245Z
4,Shirt,20,clothing,2025-10-26T02:46:53.245Z
5,Trosure,30,clothing,2025-10-26T13:43:55.307Z


In [0]:
from delta.tables import DeltaTable

if len(dbutils.fs.ls("/Volumes/pyspark_cata/source/db_volume/products_sink")) > 0:

    dlt_obj = DeltaTable.forPath(spark,"/Volumes/pyspark_cata/source/db_volume/products_sink/")

    dlt_obj.alias("trg").merge(
        df.alias("src"),
        "src.id = trg.id")\
        .whenMatchedUpdateAll(condition="src.updatedDate >= trg.updatedDate")\
        .whenNotMatchedInsertAll()\
        .execute()
    print("This is upserting now")

else:
    df.write.format("delta")\
        .mode("Overwrite")\
        .save("/Volumes/pyspark_cata/source/db_volume/products_sink")

This is upserting now


In [0]:
%sql
SELECT * FROM delta.`/Volumes/pyspark_cata/source/db_volume/products_sink/`

id,name,price,category,updatedDate
1,iPhone,1000,electronics,2025-10-26T02:46:53.245Z
2,Macbook,2000,electronics,2025-10-26T02:46:53.245Z
3,T-Shirt,10,clothing,2025-10-26T02:46:53.245Z
4,Shirt,20,clothing,2025-10-26T02:46:53.245Z
5,Trosure,30,clothing,2025-10-26T13:43:55.307Z
